In [1]:
import sys
sys.path.append("../")
import ScheduleFlow

import numpy as np

In [2]:
num_processing_units = 10

In [3]:
# create workload
def create_job_list(num_processing_units, num_jobs):
    job_list = set()

    for i in range(num_jobs):
        execution_time = np.random.randint(11, 100)
        request_time = execution_time + int(i / 2) * 10
        processing_units = np.random.randint(1, num_processing_units + 1)
        submission_time = 0
        job_list.add(ScheduleFlow.Application(
            processing_units,
            submission_time,
            execution_time,
            [request_time]))
    job_list.add(ScheduleFlow.Application(np.random.randint(9, 11), 0,
                                          100, [90, 135]))
    return job_list

job_list = create_job_list(num_processing_units, 3)
print(list(job_list)[0])
job_list

Job -1: 10 nodes; 0.0 submission time; 100.0 total execution time (90.0 requested)


{Job(Nodes: 10, Submission: 0.0, Walltime: 100.0, Request: 90.0),
 Job(Nodes: 7, Submission: 0.0, Walltime: 75.0, Request: 75.0),
 Job(Nodes: 8, Submission: 0.0, Walltime: 81.0, Request: 81.0),
 Job(Nodes: 3, Submission: 0.0, Walltime: 36.0, Request: 46.0)}

In [4]:
# create the simulator
job_list = create_job_list(num_processing_units, 10)
simulator = ScheduleFlow.Simulator(check_correctness=True,
                                   output_file_handler=sys.stdout,
                                   loops = 10)
simulator

Simulator(GIF: False, Check_correctness: True, Loops: 10, Output: <ipykernel.iostream.OutStream object at 0x7fc56f0a1080>, Jobs: 0)

In [5]:
sch = ScheduleFlow.BatchScheduler(ScheduleFlow.System(num_processing_units))
print(sch)

Batch Scheduler: System: 10 total nodes (10 currently free); 0 jobs in queue; 0 jobs running


In [6]:
simulator.create_scenario("test1", sch, job_list=job_list)
simulator

Simulator(GIF: False, Check_correctness: True, Loops: 10, Output: <ipykernel.iostream.OutStream object at 0x7fc56f0a1080>, Jobs: 11)

In [7]:
results = simulator.run(metrics=["all system"])

Scenario name : system makespan : system utilization : 
test1 : 688.00 : 0.62 : 
test1 : 688.00 : 0.62 : 
test1 : 688.00 : 0.62 : 
test1 : 688.00 : 0.62 : 
test1 : 688.00 : 0.62 : 
test1 : 688.00 : 0.62 : 
test1 : 688.00 : 0.62 : 
test1 : 688.00 : 0.62 : 
test1 : 688.00 : 0.62 : 
test1 : 688.00 : 0.62 : 


In [8]:
results

{'system makespan': 688.0, 'system utilization': 0.6247093023255815}

In [9]:
simulator = ScheduleFlow.Simulator(check_correctness=True)
simulator

Simulator(GIF: False, Check_correctness: True, Loops: 1, Output: None, Jobs: 0)

In [12]:
loops = 2
avg_results = {}
for i in range(loops):
    job_list = create_job_list(num_processing_units, 10)
    results = simulator.run_scenario(
        "test2", sch, job_list, metrics=["utilization", "response time", "wait time"])
    print(results)
    if len(avg_results)==0:
        avg_results = results
    else:
        avg_results = {i:avg_results[i]+results[i] for i in avg_results}
avg_results = {i:avg_results[i]/loops for i in avg_results}
print(avg_results)

{'job failures': 1.0, 'job response time': 319.1818181818182, 'job stretch': 9.581087472511957, 'job utilization': 0.6508398393323233, 'job wait time': 240.66666666666666, 'system makespan': 598.0, 'system utilization': 0.5568561872909699}
{'job failures': 1.0, 'job response time': 395.09090909090907, 'job stretch': 6.60229987904446, 'job utilization': 0.7299589644992353, 'job wait time': 291.75, 'system makespan': 668.0, 'system utilization': 0.6782934131736527}
{'job failures': 1.0, 'job response time': 357.1363636363636, 'job stretch': 8.091693675778208, 'job utilization': 0.6903994019157793, 'job wait time': 266.2083333333333, 'system makespan': 633.0, 'system utilization': 0.6175748002323113}
